# Homework 8

## CSCI E-82A


In the a previous homework assignments, you used two different dynamic programming algorithms and Monte Carlo reinforcement learning to solve a robot navigation problem by finding optimal paths to a goal in a simplified warehouse environment. Now you will use time differencing reinforcement learning to find optimal paths in the same environment.

The configuration of the warehouse environment is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

You are, no doubt, familiar with the representation for this problem by now.    

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [1]:
## import numpy for latter
import numpy as np
import numpy.random as nr

## Define the transition dictonary of dictionaries:
neighbors = {
    0:{'u':0, 'd':5, 'l':0, 'r':1},
    1:{'u':1, 'd':1, 'l':0, 'r':2},
    2:{'u':2, 'd':2, 'l':1, 'r':3},
    3:{'u':3, 'd':3, 'l':2, 'r':4},
    4:{'u':4, 'd':9, 'l':3, 'r':4},
    5:{'u':0, 'd':10, 'l':5, 'r':5},
    6:{'u':6, 'd':6, 'l':6, 'r':6},
    7:{'u':7, 'd':7, 'l':7, 'r':7},
    8:{'u':8, 'd':8, 'l':8, 'r':8},
    9:{'u':4, 'd':14, 'l':9, 'r':9},
    10:{'u':5, 'd':15, 'l':10, 'r':11},
    11:{'u':11, 'd':11, 'l':10, 'r':12},
    12:{'u':12, 'd':12, 'l':12, 'r':12},
    13:{'u':13, 'd':13, 'l':12, 'r':14},
    14:{'u':9, 'd':19, 'l':13, 'r':14},
    15:{'u':10, 'd':20, 'l':15, 'r':15},
    16:{'u':16, 'd':16, 'l':16, 'r':16},
    17:{'u':17, 'd':17, 'l':17, 'r':17},
    18:{'u':18, 'd':18, 'l':18, 'r':18},
    19:{'u':14, 'd':24, 'l':19, 'r':19},
    20:{'u':15, 'd':20, 'l':20, 'r':21},
    21:{'u':21, 'd':21, 'l':20, 'r':22},
    22:{'u':22, 'd':22, 'l':21, 'r':23},
    23:{'u':23, 'd':23, 'l':22, 'r':24},
    24:{'u':19, 'd':24, 'l':23, 'r':24}
}


You need to define the initial transition probabilities for the Markov process. Set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the RL problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the DP algorithm is to learn the transition policy.

In [2]:
initial_policy = {
    0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    15:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    16:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    17:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    18:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    19:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    20:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    21:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    22:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    23:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
    24:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}
}

The robot receives the following rewards:
- 10 for entering position 0. 
- -1 for attempting to leave the grid. In other words, we penalize the robot for hitting the edges of the grid.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another. If we did not have this penalty, the robot could follow any random plan to the goal which did not hit the edges. 

This **reward structure is unknown to the MC RL agent**. The agent must **learn** the rewards by sampling the environment. 

In the code cell below encode your representation of this reward structure you will use in your simulated environment.  

In [3]:
rewards = {
    0:{'u':-1.0, 'd':-0.1, 'l':-1.0, 'r':-0.1},
    1:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':-0.1},
    2:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':-0.1},
    3:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':-0.1},
    4:{'u':-1.0, 'd':-0.1, 'l':-0.1, 'r':-1.0},
    5:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-1.0},
    6:{'u':0, 'd':0, 'l':0, 'r':0},
    7:{'u':0, 'd':0, 'l':0, 'r':0},
    8:{'u':0, 'd':0, 'l':0, 'r':0},
    9:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-1.0},
    10:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-0.1},
    11:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':10.0},
    12:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
    13:{'u':-1.0, 'd':-1.0, 'l':10.0, 'r':-0.1},
    14:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1.0},
    15:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-1.0},
    16:{'u':0, 'd':0, 'l':0, 'r':0},
    17:{'u':0, 'd':0, 'l':0, 'r':0},
    18:{'u':0, 'd':0, 'l':0, 'r':0},
    19:{'u':-0.1, 'd':-0.1, 'l':-1.0, 'r':-1.0},
    20:{'u':-0.1, 'd':-1.0, 'l':-1.0, 'r':-0.1},
    21:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':-0.1},
    22:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':-0.1},
    23:{'u':-1.0, 'd':-1.0, 'l':-0.1, 'r':-0.1},
    24:{'u':-0.1, 'd':-1.0, 'l':-0.1, 'r':-1.0}
}

You will find it useful to create a list of taboo states, which you can encode in the cell below.

In [4]:
taboo_states = [6, 7, 8, 16, 17, 18]

## TD(0) Policy Evaluation

With your representations defined, you can now create and test functions to perform TD(0) **policy evaluation**. 

As a first step you will need a function to find the rewards and next state given a state and an action. You are welcome to start with the `state_values` function from the TD/Q-learning notebook. However, keep in mind that you must modify this code to correctly treat the taboo states of the barrier. Specifically, taboo states should not be visited. 

Execute your code to test it for each possible action from state 11.  

In [5]:
def is_terminal(state, terminal = 12):
    return state == terminal

def simulate_environment(s, action, neighbors = neighbors, rewards = rewards, terminal = 12):
    """
    Function simulates the environment
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]
    reward = rewards[s][action]
    if s_prime in taboo_states:
        return (s, reward, is_terminal(s, terminal))
    return (s_prime, reward, is_terminal(s_prime, terminal))

def take_action(state, policy, actions = {1:'u', 2:'d', 3:'l', 4:'r'}):
    '''Function takes action given state using the transition probabilities 
    of the policy'''
    ## Find the action given the transistion probabilities defined by the policy.
    action = actions[nr.choice(range(len(actions)), p = list(policy[state].values())) + 1]
    s_prime, reward, terminal = simulate_environment(state, action)
    return (action, s_prime, reward, terminal)

def start_episode(n_states):
    '''Function to find a random starting value for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state) or state in taboo_states):
        state = nr.choice(range(n_states))
    return state

# Test the function
for a in ['u', 'd', 'r', 'l']:
    print(simulate_environment(11, a))



(11, -1.0, False)
(11, -1.0, False)
(12, 10.0, True)
(10, -0.1, False)


Examine your results. Are the action values consistent with the transitions?

ANS: Yes

Next, you need to create a function to compute the state values using the TD(0) algorithm. You should use the function you just created  to find the rewards and next state given a state and action. You are welcome to use the `td_0_state_values` function from the TD/Q-learning notebook as a starting point.  

Execute your function for 1,000 episodes and examine the results.

In [6]:
def td_0_state_values(policy, n_samps, alpha = 0.2, gamma = 1.0):
    """
    Function for TD(0) policy evalutation
    """
    
    ## Find the starting state
    n_states = len(policy)
    current_state = start_episode(n_states)
    terminal = False
    
    ## Array for state values
    v = np.zeros((n_states,1))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action, s_prime, reward, terminal = take_action(current_state, policy)
        ## Compute the TD error
        delta = reward + gamma*v[s_prime] - v[current_state]
        ## Update the state value
        v[current_state] = v[current_state] + alpha*delta
        current_state = s_prime
        if(terminal): ## start new episode when terminal
            current_state = start_episode(n_states)
    return(v)

td_0_state_values(initial_policy, 1000).reshape((5,5))

array([[-6.04544377, -6.64406109, -5.84384902, -5.44237094, -5.88334182],
       [-4.21291145,  0.        ,  0.        ,  0.        , -5.49114745],
       [-1.93522922,  2.46518347,  0.        ,  0.62936597, -3.97710824],
       [-3.61483869,  0.        ,  0.        ,  0.        , -4.65099312],
       [-4.7488387 , -4.83463173, -4.94295907, -5.09630443, -5.15714986]])

Examine your results and answer the following questions to ensure you action value function operates correctly:
1. Are the values of the taboo states 0? ANS: Yes
2. Are the states with the highest values adjacent to the terminal state? ANS: Yes
3. Are the values of the states decreasing as the distance from the terminal state increases? ANS: Yes


## SARSA(0) Policy Improvement

Now you will perform policy improvement using the SARSA(0) algorithm.  You are welcome to start with the `select_a_prime` and `SARSA_0` functions from the TD/Q-learning notebooks.    

Execute your code for 1,000 episodes, and with $\alpha = 0.2$, and $\epsilon = 0.1$)

In [7]:
import pandas as pd

def print_Q(Q):
    Q = pd.DataFrame(Q, columns = ['up', 'down', 'left', 'right'])
    print(Q)

def new_episode(n_states, policy):
    '''This function provides a start for a TD
    episode making sure the first transition is not 
    the termnal state'''
    current_state = start_episode(n_states)
    ## Find fist action and reward
    action, s_prime, reward, terminal = take_action(current_state, policy)
    return(current_state, action, s_prime, reward, terminal)    


def SARSA_0(policy, n_samps, alpha = 0.1, gamma = 0.9, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function for TD(0) policy evalutation
    """
    
    ## Find the starting state
    n_states = len(policy)
    current_state, action, s_prime, reward, terminal = new_episode(n_states, policy)
    action_idx = action_index[action]
    
    ## Array for state values
    q = np.zeros((n_states, len(policy[0])))
    
    for _ in range(n_samps):
        ## Find the next action and reward
        action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(s_prime, policy)
        action_idx_prime = action_index[action_prime]
        ## Compute the TD error
        delta = reward + gamma*q[s_prime, action_idx_prime] - q[current_state, action_idx]
        ## Update the action values
        q[current_state, action_idx] = q[current_state, action_idx] + alpha*delta
        ## Update the state, action and reward for the next time step
        current_state = s_prime
        s_prime = s_prime_prime
        action = action_prime
        reward = reward_prime
        terminal = terminal_prime
        action_idx = action_idx_prime

        ## Check if end of episode
        if(terminal): 
            ## start new episode
            current_state, action, s_prime, reward, terminal = new_episode(n_states, policy)        
    return(q)


Q = SARSA_0(initial_policy, 1000, alpha = 0.2, gamma = 0.1)
print_Q(Q)

          up      down      left     right
0  -1.035995 -0.161704 -1.045391 -0.160261
1  -1.069659 -1.059911 -0.157360 -0.137088
2  -1.008263 -0.959963 -0.127110 -0.174727
3  -1.049064 -1.050818 -0.138498 -0.151599
4  -1.049935 -0.144062 -0.133155 -1.059425
5  -0.174848 -0.138995 -0.983999 -0.945098
6   0.000000  0.000000  0.000000  0.000000
7   0.000000  0.000000  0.000000  0.000000
8   0.000000  0.000000  0.000000  0.000000
9  -0.160813 -0.103748 -1.007956 -0.921042
10 -0.125906 -0.142587 -1.014071 -0.105243
11 -0.607595 -0.745366 -0.094925  0.000000
12  0.000000  0.000000  0.000000  0.000000
13 -0.207200 -0.741218  0.000000 -0.117928
14 -0.130681 -0.153681 -0.156559 -0.881425
15 -0.136015 -0.148030 -1.038070 -1.038472
16  0.000000  0.000000  0.000000  0.000000
17  0.000000  0.000000  0.000000  0.000000
18  0.000000  0.000000  0.000000  0.000000
19 -0.122371 -0.131547 -1.036379 -1.060449
20 -0.145084 -1.049751 -1.073560 -0.128132
21 -0.862921 -0.920943 -0.119333 -0.108367
22 -0.81725

Examine the action values you have computed. Ensure that the action values are 0 for the goal and taboo states. Also check that the actions with the largest values for each state make sense in terms of reaching the goal. 

With the action value function completed, you will now create and test code to perform GPI with SARSA(0).  You are welcome to use the `SRASA_0_GPI` function from the TD/Q-learning notebook as a starting point. 

Execute your code for 10 cycles of 100 episodes, with $\alpha = 0.2$, $\gamma = 0.9$ and $\epsilon = 0.01$, and examine the results.

In [8]:
def update_policy(policy, Q, epsilon, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    '''Updates the policy based on estiamtes of Q using 
    an epslion greedy algorithm. The action with the highest
    action value is used.'''
    
    ## Find the keys for the actions in the policy
    keys = list(policy[0].keys())
    
    ## Iterate over the states and find the maximm action value.
    for state in range(len(policy)):
        ## First find the index of the max Q values  
        q = Q[state,:]
        max_action_index = np.where(q == max(q))[0]

        ## Find the probabilities for the transitions
        n_transitions = float(len(q))
        n_max_transitions = float(len(max_action_index))
        p_max_transitions = (1.0 - epsilon *(n_transitions - n_max_transitions))/(n_max_transitions)
  
        ## Now assign the probabilities to the policy as epsilon greedy.
        for key in keys:
            if(action_index[key] in max_action_index): policy[state][key] = p_max_transitions
            else: policy[state][key] = epsilon
    return(policy) 


def SARSA_GPI(policy, n_samples, n_cycles, epsilon = 0.01, n_actions = 4):
    '''Function perfoms GPI using Monte Carlo value estimation.
    Updates to policy are epsilon greedy to prevent the algorithm
    from being trapped at some point.'''
    Q = np.zeros((len(policy), n_actions))
    ## Iterate over the required number of cycles
    for _ in range(n_cycles):
        Q = SARSA_0(policy, n_samples, alpha = 0.2, gamma = 0.9)
        policy = update_policy(policy, Q, epsilon = epsilon)
    return(policy)

improved_policy = SARSA_GPI(initial_policy, 100, 10, epsilon = 0.01)  
for state in range(25):
    print(improved_policy[state])

{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.33, 'd': 0.33, 'l': 0.33, 'r': 0.01}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.97, 'd': 0.01, 'l': 0.01, 'r': 0.01}
{'u': 0.49, 'd': 0.49, 'l': 0.01, 'r': 0.01}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.01, 'd': 0.01, 'l': 0.97, 'r': 0.01}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.01, 'd': 0.97, 'l': 0.01, 'r': 0.01}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25, 'd': 0.25, 'l': 0.25, 'r': 0.25}
{'u': 0.25

Verify that your results make sense? For example, starting at state 2 or 22, do the most probable actions follow a shortest path?

ANS: Yes

## Apply Double Q-Learning

As a next step, you will apply Double Q-learning(0) to the warehouse navigation problem. In the cell below create and test a function to perform Double Q-Learning for this problem. You are welcome to use the `double_Q_learning` function from the TD/Q-learning notebook as a starting point.

Execute your code for 10 cycles of 500 episodes, with $\alpha = 0.2$, and $\gamma = 0.9$ and examine the results.

In [9]:
def action_lookup(index):
    """Helper function returns action given an index"""
    action_dic = {0:'u', 1:'d', 2:'l', 3:'r'}
    return action_dic[index]


def simulate_environment_q(s, action, neighbors = neighbors, rewards = rewards, terminal = 12):
    """
    Function simulates the environment for Q-learning.
    returns s_prime and reward given s and action
    """
    s_prime = neighbors[s][action]        
    reward_prime = np.array([rewards[s_prime][a] for a in rewards[0].keys()])
    if s_prime in taboo_states:
        return (s, reward_prime, is_terminal(s, terminal))
    return (s_prime, reward_prime, is_terminal(s_prime, terminal))

def start_episode_q(n_states, n_actions):
    '''Function to find a random starting values for the episode
    that is not the terminal state'''
    state = nr.choice(range(n_states))
    while(is_terminal(state) or state in taboo_states):  ## Make sure not starting at the terminal state
         state = nr.choice(range(n_states))
    ## Now find a random starting action index
    a_index = nr.choice(range(4), size = 1)[0]
    s_prime, reward, terminal = simulate_environment_q(state, action_lookup(a_index))
    return state, a_index, reward[a_index] ## action_lookup(a_index), reward[a_index]

def update_double_Q(q1, q2, current_state, a_index, reward, alpha, gamma):
    """Function to update the actions values in the Q matrix"""
    ## Get s_prime given s and a
    s_prime, reward_prime, terminal = simulate_environment_q(current_state, action_lookup(a_index))
    a_prime_index = nr.choice(np.where(reward_prime == max(reward_prime))[0], size = 1)[0]
    ## Update the action values 
    q1[current_state,a_index] = q1[current_state,a_index] + alpha * (reward + gamma * (q2[s_prime,a_prime_index] - q1[current_state,a_index]))
    return q1, s_prime, reward_prime, terminal, a_prime_index


def double_Q_learning_0(policy, episodes, alpha = 0.2, gamma = 0.9):
    """
    Function to perform Q-learning(0) control policy improvement.
    """
    ## Initialize the state list and action values
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    
    ## Initialize both Q matricies
    Q1 = np.zeros((n_states,n_actions))
    Q2 = np.zeros((n_states,n_actions))
    
    for _ in range(episodes): # Loop over the episodes
        terminal = False
        ## Find the inital state, action index and reward
        current_state, a_index, reward = start_episode_q(n_states,n_actions)
        
        while(not terminal): # Episode ends where get to terminal state   
            ## Update the action values in Q1 or Q2 based on random choice
            if(nr.uniform() <= 0.5):
                Q1, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q1, Q2, current_state, a_index, reward, alpha, gamma)
            else:
                Q2, s_prime, reward_prime, terminal, a_prime_index = update_double_Q(Q2, Q1, current_state, a_index, reward, alpha, gamma)
            ## Set action, reward and state for next iteration
            a_index = a_prime_index
            current_state = s_prime
            reward = reward_prime[a_prime_index]
    return(Q1)

Q_ = double_Q_learning_0(initial_policy, 500)
print_Q(Q_)

          up      down       left      right
0   1.074551  6.393472   1.311252   4.115865
1   1.434339 -0.230652   4.763815   3.889472
2   1.314341 -0.274580   4.286845   4.446001
3   0.548085 -0.107054   3.950863   4.644905
4   0.477104  5.588211   4.313492   0.689493
5   4.766605  8.851764  -0.269621   2.346641
6   0.000000  0.000000   0.000000   0.000000
7   0.000000  0.000000   0.000000   0.000000
8   0.000000  0.000000   0.000000   0.000000
9   4.423793  5.708195   0.890611  -0.137103
10  5.830583  5.760038   4.218673  11.856163
11  1.799730  6.135114   2.388766  11.111094
12  0.000000  0.000000   0.000000   0.000000
13  7.142131  5.683015  10.833461   0.545310
14  5.488866  5.423136  10.879625   1.579511
15  6.306114  4.339178   0.922682   1.621455
16  0.000000  0.000000   0.000000   0.000000
17  0.000000  0.000000   0.000000   0.000000
18  0.000000  0.000000   0.000000   0.000000
19  8.065667  3.502538   0.596186   0.198271
20  5.694021  0.497408   2.378138   4.029940
21  1.0046

Examine the action values you have computed. Ensure that the action values are 0 for the goal and taboo states. Also check that the actions with the largest values for each state make sense in terms of reaching the goal. 

With the action value function completed, you will now create and test code to perform GPI with Double Q-Learning(0).  You are welcome to use the `double_Q_learning_0_GPI` function from the TD/Q-learning notebook as a starting point. 

Execute your code for 10 cycles of 500 episodes, with $\alpha = 0.2$, $\gamma = 0.9$ and $\epsilon = 0.01$, and examine the results. 

In [10]:
import copy

def double_Q_learning_0_GPI(policy, neighbors, reward, cycles, episodes, goal, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    ## iterate over GPI cycles
    current_policy = copy.deepcopy(policy)
    for _ in range(cycles):
        ## Evaluate policy with SARSA
        Q = double_Q_learning_0(policy, episodes, alpha, gamma)
        
        for s in list(current_policy.keys()): # iterate over all states
            ## Find the index action with the largest Q values 
            ## May be more than one. 
            max_index = np.where(Q[s,:] == max(Q[s,:]))[0]

            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(Q.shape[0])
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder   
                    
    return(current_policy)                    
 

Double_Q_0_Policy = double_Q_learning_0_GPI(initial_policy, neighbors, rewards, cycles = 10, episodes = 500, goal = 12, alpha = 0.2, epsilon = 0.01)
print(Double_Q_0_Policy)

{0: {'u': 0.01, 'd': 0.76, 'l': 0.01, 'r': 0.01}, 1: {'u': 0.01, 'd': 0.01, 'l': 0.76, 'r': 0.01}, 2: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.76}, 3: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.76}, 4: {'u': 0.01, 'd': 0.76, 'l': 0.01, 'r': 0.01}, 5: {'u': 0.01, 'd': 0.76, 'l': 0.01, 'r': 0.01}, 6: {'u': 0.04000000000000001, 'd': 0.04000000000000001, 'l': 0.04000000000000001, 'r': 0.04000000000000001}, 7: {'u': 0.04000000000000001, 'd': 0.04000000000000001, 'l': 0.04000000000000001, 'r': 0.04000000000000001}, 8: {'u': 0.04000000000000001, 'd': 0.04000000000000001, 'l': 0.04000000000000001, 'r': 0.04000000000000001}, 9: {'u': 0.01, 'd': 0.76, 'l': 0.01, 'r': 0.01}, 10: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.76}, 11: {'u': 0.01, 'd': 0.01, 'l': 0.01, 'r': 0.76}, 12: {'u': 0.04000000000000001, 'd': 0.04000000000000001, 'l': 0.04000000000000001, 'r': 0.04000000000000001}, 13: {'u': 0.01, 'd': 0.01, 'l': 0.76, 'r': 0.01}, 14: {'u': 0.01, 'd': 0.01, 'l': 0.76, 'r': 0.01}, 15: {'u': 0.76, 'd'

Verify that your results make sense? For example, starting at state 2 or 22, do the most probable actions follow a shortest path?

ANS: Yes

## N-Step TD Learning

Finally, you will apply N-Step TD learning and N-Step SARSA to the warehouse navigation problem.  First create a function to perform N-step TD policy evaluation. You are welcome to start with the `TD_n` policy evaluation function from the TD/Q-Learning notebook. 

Test your function using 1,000 episodes, $n = 4$, $\gamma = 0.9$, and $\alpha = 0.2$.

In [13]:
def TD_n(policy, episodes, n, alpha = 0.2, gamma = 0.9, epsilon = 0.1, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function to perform TD(N) policy evaluation.
    """
    ## Initialize the state list and action values
    # action_index = list(range(len(list(policy[0].keys()))))
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    v = np.zeros((n_states))
    
    for _ in range(episodes):
        ## Initialize variables
        T = float("inf")
        tau = 0
        t = 0
        rewards = []
       
        ## Get the random initial state
        current_state = start_episode(n_states)   
        state = [current_state]
        ## Initial action
        action, s_prime, reward, terminal = take_action(current_state, policy)
        state.append(s_prime)
        
        while(not (tau == T - 1)):
            if(t < T):
                ## Append the reward to the list
                rewards.append(reward)             
                if(terminal): 
                    ## update T if at terminal state
                    T = t + 1
                else: 
                    ## Get the next action state and rewards
                    action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(current_state, policy)
                    state.append(s_prime_prime)
                      
            ## Update tau
            tau = t - n + 1
            
            if(tau > 0):
                G = 0.0
                for i in range(tau + 1, min(tau + n, T)):
                    exponent = i + tau - 1
                    G = G + gamma**exponent * rewards[i]
                if(tau + n < T): G += gamma**n * v[state[tau + n]] 
                v[state[tau]] = v[state[tau]] + alpha * (G - v[state[tau]])    
                
            
            ## Update variables for the next step
            t += 1
            current_state = s_prime

            if(not terminal):
                action = action_prime
                s_prime = s_prime_prime
                reward = reward_prime
                terminal = terminal_prime
    return(v)     

TD_n(initial_policy, 1000, 4).reshape((5,5))

array([[-2.76881266e-04, -3.99686431e-04, -8.09754984e-04,
        -2.33683905e-03, -8.67719338e-03],
       [-2.74633649e-04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -2.55590876e-02],
       [-1.13231224e-04, -2.30275686e-03,  0.00000000e+00,
         1.30343423e-02,  5.38239667e-02],
       [-1.23708298e-04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -5.32907519e-02],
       [-6.75005319e-03, -3.40951973e-02, -1.64927121e-01,
        -1.08020006e-01, -3.97212532e-02]])

Verify that the result you obtained appears correct. Are the values of the goal and taboo states all 0? Do the state values decrease with distance from the goal? Yes

Now that you have an estimate of the best values for the number of steps and the learning rate you can compute the action values using multi-step SARSA. In the cell below, create and test a function to compute the action values using N-step SARSA. You are welcome to use the `SRARSA_n` function from the TD/Q-learning notebook as a starting point. 

Test your function by executing 4-step SARSA for 1,000 episodes with $\alpha = 0.2$ and $\gamma = 0.9$ and using the optimum number of steps and learning rate you have determined. 

In [16]:
def SARSA_n(policy, episodes, n, alpha = 0.2, gamma = 0.9, epsilon = 0.1, action_index = {'u':0, 'd':1, 'l':2, 'r':3}):
    """
    Function to perform TD(N) policy evaluation.
    """
    ## Initialize the state list and action values
#     action_index = list(range(len(list(policy[0].keys()))))
    states = list(policy.keys())
    n_states = len(states)
    n_actions = len(policy[0].keys())
    q = np.zeros((n_states, n_actions))
    
    for _ in range(episodes):
        ## Initialize variables
        T = float("inf")
        tau = 0
        t = 0
        rewards = []
       
        ## Get the random initial state
        current_state = start_episode(n_states)   
        state = [current_state]
        ## Initial action
        action, s_prime, reward, terminal = take_action(current_state, policy)
        state.append(s_prime)
        
        while(not (tau == T - 1)):
            if(t < T):
                ## Append the reward to the list
                rewards.append(reward)             
                if(terminal): 
                    ## update T if at terminal state
                    T = t + 1
                else: 
                    ## Get the next action state and rewards
                    action_prime, s_prime_prime, reward_prime, terminal_prime = take_action(current_state, policy)
                    state.append(s_prime_prime)
                      
            ## Update tau
            tau = t - n + 1
            
            if(tau > 0):
                G = 0.0
                for i in range(tau + 1, min(tau + n, T)):
                    exponent = i + tau - 1
                    G = G + gamma**exponent * rewards[i]
                if(tau + n < T):
                    G += gamma**n * q[state[tau + n], action_index[action_prime]] 
                q[state[tau], action_index[action]] = q[state[tau], action_index[action]] + alpha * (G - q[state[tau], action_index[action]])    
                
            
            ## Update variables for the next step
            t += 1
            current_state = s_prime
            if(not terminal):
                action = action_prime
                s_prime = s_prime_prime
                reward = reward_prime
                terminal = terminal_prime
    return(q)              
            
Q = SARSA_n(initial_policy, 1000, 4)
print_Q(Q)

          up      down      left     right
0  -0.022118 -0.014131 -0.024308 -0.025394
1  -0.021567 -0.017823 -0.018544 -0.024287
2  -0.025851 -0.019119 -0.020298 -0.009010
3  -0.008256 -0.008216 -0.021135 -0.004191
4  -0.007465 -0.008087 -0.015527 -0.006827
5  -0.021421 -0.009458 -0.025437 -0.015158
6   0.000000  0.000000  0.000000  0.000000
7   0.000000  0.000000  0.000000  0.000000
8   0.000000  0.000000  0.000000  0.000000
9  -0.007061 -0.002219 -0.008068 -0.003262
10 -0.015452 -0.002098 -0.003754 -0.008367
11 -0.005239 -0.005189 -0.011692  0.065345
12  0.000000  0.000000  0.000000  0.000000
13 -0.079683 -0.074639  0.082011 -0.012479
14 -0.030278 -0.008845  0.116296 -0.022869
15 -0.010180 -0.002362 -0.001154 -0.001242
16  0.000000  0.000000  0.000000  0.000000
17  0.000000  0.000000  0.000000  0.000000
18  0.000000  0.000000  0.000000  0.000000
19 -0.008601 -0.004456 -0.007046 -0.006806
20 -0.001993 -0.001157 -0.001529 -0.002660
21 -0.001263 -0.001314 -0.000855 -0.003512
22 -0.00206

Verify that the results you have computed appear correct using the aforementioned criteria. 

Finally, create a function to use the GPI algorithm with N-step SARSA in the cell below. You are welcome to start with the `SARSA_n_GPI` function from the TD/Q-learning notebook. 

Execute your function using 4 step SARSA for 5 cycles of 500 episodes, with $\alpha = 0.2$, $\epsilon = 0.1$, and $\gamma = 0.9$.

In [ ]:
import copy

def SARSA_n_GPI(policy, n, cycles, episodes, alpha = 0.2, gamma = 0.9, epsilon = 0.1):
    ## iterate over GPI cycles
    current_policy = copy.deepcopy(policy)
    for _ in range(cycles):
        ## Evaluate policy with SARSA
        Q = SARSA_n(policy, episodes, n, alpha, gamma, epsilon)

        for s in list(current_policy.keys()): # iterate over all states
            ## Find the index action with the largest Q values 
            ## May be more than one. 
            max_index = np.where(Q[s,:] == max(Q[s,:]))[0]
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(Q.shape[0])
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder   
                    
    return(current_policy)                    

SARSA_N_Policy = SARSA_n_GPI(initial_policy, 4, 5, 500, 0.2, 0.9, 0.1)
print(SARSA_N_Policy)

Examine your results. Verify that the most probable paths to the goal from states 2 and 22 are the shortest possible.  